### Scraping MLB stats from ESPN go###

In this homework we will try to analyze the batting performances of different teams in Major League Baseball using the data available in the following link http://espn.go.com/mlb/stats/team/_/stat/batting 

We will first scrape the page corresponding to the 2015 season 

In [1]:
url="http://espn.go.com/mlb/stats/team/_/stat/batting/year/2015"

We will now scrape the table found in the above link. You can follow the class notebook to understand how to capture html table tags.

Q1) Write a function which will take the above url and return a pandas dataframe corresponding to the table found in the given link. 
-Note from Sachin: only keep the information from the teams

In [ ]:
#import pandas
#pandas.read_csv("http://espn.go.com/mlb/stats/team/_/stat/batting/year/2015")

In [2]:
import requests
import lxml.html as lh
page = requests.get(url)
doc = lh.fromstring(page.content)
page.content[:500]

'<!DOCTYPE html>\n<html xmlns:fb="http://www.facebook.com/2008/fbml">\n<head><script src="http://sports-ak.espn.go.com/sports/optimizely.js"></script><meta charset="iso-8859-1">\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<link rel="icon" sizes="any" mask href="http://a.espncdn.com/prod/assets/icons/E.svg">\n<meta name="theme-color" content="#CC0000">\n<script type="text/javascript">    \n    if(true && navigator && navigator.userAgent.toLowerCase().indexOf("teamstream") >= 0) {\n   '

In [3]:
#tr_elements stands for table row

tr_elements = doc.xpath('//tr')
[len(T) for T in tr_elements[:20]]

[1, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15]

In [4]:
for i in range(len(tr_elements)):
    if len(tr_elements[i])==15:
        print i, tr_elements[i].text_content()

1 RKTEAMGPABRH2B3BHRTBRBIAVGOBPSLGOPS
2 1Toronto16255098911480308172322518852.269.340.457.797
3 2NY Yankees16255677641397272192122343737.251.323.421.744
4 3Texas16255117511419279321722278707.257.325.413.739
5 4Boston16256407481496294331612339706.265.325.415.740
6 5Colorado16255727371479274491862409702.265.315.432.748
7 6Houston16254597291363278262302383691.250.315.437.752
8 7Kansas City16255757241497300421392298689.269.322.412.734
9 8Arizona16256497201494289481542341680.264.324.414.738
10 9Baltimore16254857131370246202172307686.250.307.421.728
11 10Washington16254287031363265131772185665.251.321.403.724
12 11Pittsburgh16256316971462292271402228661.260.323.396.719
13 12Minnesota16254676961349277441562182661.247.305.399.704
14 13San Francisco16255656961486288391362260663.267.326.406.732
15 14Oakland16256006941405277461462212661.251.312.395.707
16 15Detroit16156056891515289491512355660.270.328.420.748
17 16Chicago Cubs16254916891341272301712186657.244.321.398.719
18 17NY Mets1625527683135

In [11]:
col=[]  # collect column names into col
T=tr_elements[15]
print type(T)
i=0
print len(T)
for t in T.iterchildren():
    i+=1
    name=t.text_content()
    print '%d:"%s"'%(i,name)
    col.append((name,[]))

<class 'lxml.html.HtmlElement'>
15
1:"14"
2:"Oakland"
3:"162"
4:"5600"
5:"694"
6:"1405"
7:"277"
8:"46"
9:"146"
10:"2212"
11:"661"
12:".251"
13:".312"
14:".395"
15:".707"


In [12]:
for j in range(7,len(tr_elements)):
    T=tr_elements[j]
    if len(T)!=15:
        break
    i=0
    for t in T.iterchildren():
        data=t.text_content()
        if i!=1:
            try:
                data=float(data)
            except:
                print data,'cannot be converted to float, row,col=',j,i
                data=None
        col[i][1].append(data)
        i+=1
        
        
        
[len(C) for (title,C) in col]
min_len=min([len(C) for (title,C) in col])
Dict={title:column for (title,column) in col}
import pandas as pd
df=pd.DataFrame(Dict)
print df.head()

    .251   .312   .395   .707  14  1405  146  162  2212  277  46  5600  661  \
0  0.250  0.315  0.437  0.752   6  1363  230  162  2383  278  26  5459  691   
1  0.269  0.322  0.412  0.734   7  1497  139  162  2298  300  42  5575  689   
2  0.264  0.324  0.414  0.738   8  1494  154  162  2341  289  48  5649  680   
3  0.250  0.307  0.421  0.728   9  1370  217  162  2307  246  20  5485  686   
4  0.251  0.321  0.403  0.724  10  1363  177  162  2185  265  13  5428  665   

   694      Oakland  
0  729      Houston  
1  724  Kansas City  
2  720      Arizona  
3  713    Baltimore  
4  703   Washington  


In [ ]:
def SCRAPE_ESPN_MLB_STATS(url):
    
    raise "TODO"

Run the above function to scrape season 2015 stats.

In [ ]:
df_2015 = SCRAPE_ESPN_MLB_STATS(url)

We will now produce plots analyzing performance of different teams on different statistical parameters

Q2) Write a function which will take the above dataframe and a list of column names as input and produces a set of plots corresponding to each of the column names provided

In [ ]:
def produce_plots(df, col_names):
    raise "TODO"

Call the above function for the columns

1. HR: Home Runs
2. TB: Total Bases
3. RBI: Runs Batted In

Q3) We will now use the above functions to scrape for more seasons and analyze the performances over a period of 6 years from 2010-2015

In [ ]:
dfs={}
for year in xrange(0,6):
    link = 'http://espn.go.com/mlb/stats/team/_/stat/batting/year/201'+str(year)
    dfs[year]=SCRAPE_ESPN_MLB_STATS(link)

Inorder to analyze performance of teams across seasons, we will need all the data in a single dataframe. 

Q4) Use appropriate pandas functions to combine the above dictionary of year:dataframe to produce one dataframe which has a new column corresponding to the year/season 

In [ ]:
combined_df = #combine dfs dictionary with a new column for year

Q5) Now write a function which will take the above dataframe and a list of column names and produces a set of plots corresponding to each of the columns provided. Each plot is a set of subplots, where every subplot is a line graph of the column values over the 6 years for each of the teams in the dataframe.

In [ ]:
def produce_plots_over_seasons(combined_df, col_names):
    raise "TODO"

Call the above function for the columns 

1. HR: Home Runs
2. TB: Total Bases
3. RBI: Runs Batted In